# Análisis Exploratorio de Datos (EDA)

Este cuaderno realiza un EDA básico sobre un dataset hipotético de compras. El análisis se enfoca en preguntas de negocio sencillas y aplica métodos de filtrado y agrupación para calcular métricas relevantes.

In [4]:
!wget https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/shopping_trends.csv

--2025-03-01 13:14:21--  https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/shopping_trends.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453248 (443K) [text/plain]
Saving to: ‘shopping_trends.csv’

shopping_trends.csv 100%[===================>] 442.62K  --.-KB/s    in 0.04s   

2025-03-01 13:14:22 (9.88 MB/s) - ‘shopping_trends.csv’ saved [453248/453248]



## 1. Importar librerías y cargar datos

En este ejemplo, se asume que el archivo CSV se llama `datos_compras.csv`. Ajusta el nombre o la ruta del archivo según corresponda.

In [7]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('datos_compras.csv')

# Mostrar las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


Observamos que nuestro DataFrame contiene, entre otras, las siguientes columnas:

- **Customer ID**: ID único de cada cliente.
- **Age**: Edad del cliente.
- **Gender**: Género del cliente (Male/Female).
- **Item Purchased**: Nombre del producto comprado.
- **Category**: Categoría del producto (Footwear, Clothing, Accessories, etc.).
- **Purchase Amount (USD)**: Monto de la compra en dólares.
- **Location**: Ubicación/Estado.
- **Size**: Talla del producto.
- **Color**: Color del producto.
- **Season**: Temporada (Summer, Winter, etc.).
- **Review Rating**: Calificación de reseña (escala 1-5).
- **Subscription Status**: Indica si el cliente está suscrito (Yes/No).
- **Shipping Type**: Tipo de envío (Express, Store Pickup, etc.).
- **Discount Applied**: Indica si se aplicó descuento (Yes/No).
- **Promo Code Used**: Indica si se usó código promocional (Yes/No).
- **Previous Purchases**: Número de compras anteriores.
- **Payment Method**: Método de pago (Credit Card, Debit Card, PayPal, etc.).
- **Frequency of Purchases**: Frecuencia de compra (Daily, Weekly, Monthly, etc.).


## 2. Limpieza y revisión rápida de datos
Antes de responder a las preguntas de negocio, revisemos datos básicos: número de filas, columnas y valores nulos.

In [8]:
# Ver información general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Payment Method            3900 non-null   object 
 13  Shipping Type             3900 non-null   object 
 14  Discount

In [9]:
# Ver estadísticos descriptivos de las columnas numéricas
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.749949,25.351538
std,1125.977353,15.207589,23.685392,0.716223,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.700000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


Si encuentras valores nulos en alguna columna, podrías decidir cómo manejarlos (eliminarlos o rellenarlos). Por simplicidad, asumimos que no hay muchos datos faltantes o que no es crítico para este ejemplo.

## 3. Preguntas de negocio
A continuación, formulamos algunas preguntas de negocio y utilizamos operaciones de filtrado y/o agrupaciones para responderlas.

### 3.1 ¿Cuántos clientes únicos hay?
Esta pregunta nos ayuda a entender la base total de clientes.

In [10]:
df['Customer ID']

,Customer ID
0,1
1,2
2,3
3,4
4,5
...,...
3895,3896
3896,3897
3897,3898
3898,3899


En la base total tenemos 3899 clientes únicos.

### 3.2 ¿Cuál es el gasto promedio por género?
Para entender si hay diferencias en el monto de compra según el género, hacemos un groupby y calculamos la media.

In [11]:
df.groupby("Gender")["Purchase Amount (USD)"].median()

,Purchase Amount (USD)
Gender,
Female,60.0
Male,60.0


El gasto promedio por género fémenino y másculino es 60.

### 3.3 ¿Cuál es la calificación de reseña promedio por temporada?
Si queremos ver si las reseñas cambian según la temporada, podemos hacer una agrupación por la columna `Season` y calcular la media.

In [12]:
df.groupby("Season")["Review Rating"].median()

,Review Rating
Season,
Fall,3.7
Spring,3.8
Summer,3.7
Winter,3.7


La calificación promedio por temporada son las siguientes: otoño: 3.7, primavera: 3.8, verano: 3.7 e Invierno: 3.7.

### 3.4 ¿Cuál es el método de pago más utilizado?
Una forma rápida es usar `value_counts()` en la columna `Payment Method`.

In [13]:
df["Payment Method"].value_counts()

,count
Payment Method,
Credit Card,696
Venmo,653
Cash,648
PayPal,638
Debit Card,633
Bank Transfer,632


El método de pago más utilizado es Credit Card con 696 cuentas.

### 3.5 ¿Qué categoría se vende más?
Para saber la categoría más popular, también usamos `value_counts()`.

In [14]:
df["Category"].value_counts()

,count
Category,
Clothing,1737
Accessories,1240
Footwear,599
Outerwear,324


La categoría que más se vende es la ropa, con 1737 ventas totales.

### 3.6 ¿Cuántos clientes tienen suscripción activa?
Analizamos la columna `Subscription Status`.

In [15]:
df[df["Subscription Status"]=="Yes"]

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,1049,30,Male,Socks,Clothing,77,Maine,L,Yellow,Fall,2.7,Yes,Cash,Standard,Yes,Yes,19,Venmo,Fortnightly
1049,1050,45,Male,Sunglasses,Accessories,33,Virginia,S,Purple,Fall,2.8,Yes,Credit Card,Store Pickup,Yes,Yes,24,Cash,Quarterly
1050,1051,48,Male,Sneakers,Footwear,64,Florida,M,Beige,Winter,3.8,Yes,Credit Card,Next Day Air,Yes,Yes,19,Venmo,Monthly
1051,1052,45,Male,Socks,Clothing,53,Ohio,M,Indigo,Fall,2.6,Yes,PayPal,Free Shipping,Yes,Yes,39,PayPal,Annually


1052 clientes tienen la suscripción activa.

### 3.7 ¿Cuál es el gasto promedio por categoría?
Para entender mejor en qué categoría gastan más nuestros clientes.

In [16]:
df.groupby("Category")["Purchase Amount (USD)"].mean()

,Purchase Amount (USD)
Category,
Accessories,59.838710
Clothing,60.025331
Footwear,60.255426
Outerwear,57.172840


El gasto promedio por categoría es: accesorios: 59.83, ropa: 60.02, calzado: 60.25 y ropa de calle: 57.17.

### 3.8 ¿Cuántos clientes usan código promocional?
Podemos filtrar o contar los valores de la columna `Promo Code Used`.

In [17]:
df[df["Promo Code Used"]== "Yes"]

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,1673,18,Male,Boots,Footwear,73,South Carolina,L,Gold,Fall,3.8,No,Debit Card,Free Shipping,Yes,Yes,15,Venmo,Quarterly
1673,1674,21,Male,Blouse,Clothing,62,Hawaii,M,Violet,Fall,3.4,No,Debit Card,Next Day Air,Yes,Yes,49,Venmo,Bi-Weekly
1674,1675,35,Male,Coat,Outerwear,56,Alaska,M,Peach,Winter,3.2,No,PayPal,Free Shipping,Yes,Yes,10,Debit Card,Fortnightly
1675,1676,35,Male,Pants,Clothing,90,Colorado,M,Beige,Spring,4.7,No,Venmo,Next Day Air,Yes,Yes,21,Credit Card,Annually


1676 clientes utilizan el código promocional.

## 4. Conclusiones
- **Clientes únicos**: Nos indica cuántas personas diferentes compran en nuestra tienda.
- **Gasto promedio por género**: Sirve para comprender si existe diferencia significativa en el gasto entre hombres y mujeres.
- **Calificación promedio por temporada**: Ayuda a identificar en qué temporada los clientes suelen estar más satisfechos.
- **Métodos de pago preferidos**: Informa sobre la facilidad o preferencia de pago.
- **Categoría más vendida**: Podríamos enfocarnos en promocionarla o analizar por qué se vende más.
- **Suscripciones**: Entender cuántos clientes están suscritos para estrategias de marketing.
- **Uso de códigos promocionales**: Indica si las promociones son efectivas o no.

Este análisis permite identificar patrones y oportunidades para enfocar acciones comerciales y de marketing.